In [1]:
import pandas as pd
import glob
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf

In [3]:
from pmdarima.arima import auto_arima

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [9]:
data = pd.read_csv('consumo_mensal_energia.csv', sep=",", encoding='UTF-8', low_memory=False)
data.shape

(379870, 6)

In [10]:
data.head()

,id,referencia,cliente_id,tipo_cliente,consumo_kwh,Estado_Sigla
0,1264499,2021-08-01,21824,PJ,390.0,MG
1,1253013,2021-09-01,21824,PJ,339.0,MG
2,1089580,2021-10-01,21824,PJ,513.0,MG
3,1043501,2021-11-01,21824,PJ,500.0,MG
4,964456,2021-12-01,21824,PJ,521.0,MG


In [11]:
data['referencia'] = pd.to_datetime(data['referencia'].str.strip(), format='%Y-%m-%d')

In [12]:
data.dtypes

id                       int64
referencia      datetime64[ns]
cliente_id               int64
tipo_cliente            object
consumo_kwh            float64
Estado_Sigla            object
dtype: object

In [13]:
data.head()

,id,referencia,cliente_id,tipo_cliente,consumo_kwh,Estado_Sigla
0,1264499,2021-08-01,21824,PJ,390.0,MG
1,1253013,2021-09-01,21824,PJ,339.0,MG
2,1089580,2021-10-01,21824,PJ,513.0,MG
3,1043501,2021-11-01,21824,PJ,500.0,MG
4,964456,2021-12-01,21824,PJ,521.0,MG


In [14]:
data.columns

Index(['id', 'referencia', 'cliente_id', 'tipo_cliente', 'consumo_kwh',
       'Estado_Sigla'],
      dtype='object')

In [16]:
data = data[['referencia', 'cliente_id', 'tipo_cliente', 'consumo_kwh']]
data.head()

,referencia,cliente_id,tipo_cliente,consumo_kwh
0,2021-08-01,21824,PJ,390.0
1,2021-09-01,21824,PJ,339.0
2,2021-10-01,21824,PJ,513.0
3,2021-11-01,21824,PJ,500.0
4,2021-12-01,21824,PJ,521.0


In [17]:
a = data.groupby(['cliente_id']).agg(N = ('referencia', 'count')).reset_index()
a.head()

,cliente_id,N
0,10,22
1,46,22
2,62,8
3,180,22
4,186,22


In [18]:
a['N'].describe()

count    21810.000000
mean        17.417240
std          5.332956
min          1.000000
25%         15.000000
50%         19.000000
75%         22.000000
max         66.000000
Name: N, dtype: float64

In [19]:
a = a[(a['N'] >= 6)]
a.shape

(20662, 2)

In [21]:
data = data.merge(a, on=['cliente_id'], how='left')
data = data[data['N'].notna()]
data.shape
#(379870, 13)

(376697, 5)

In [22]:
data.head()

,referencia,cliente_id,tipo_cliente,consumo_kwh,N
0,2021-08-01,21824,PJ,390.0,22.0
1,2021-09-01,21824,PJ,339.0,22.0
2,2021-10-01,21824,PJ,513.0,22.0
3,2021-11-01,21824,PJ,500.0,22.0
4,2021-12-01,21824,PJ,521.0,22.0


In [23]:
data.tail()

,referencia,cliente_id,tipo_cliente,consumo_kwh,N
379865,2023-01-01,39822,PJ,4771.0,18.0
379866,2023-02-01,39822,PJ,4910.0,18.0
379867,2023-03-01,39822,PJ,5672.0,18.0
379868,2023-04-01,39822,PJ,4986.0,18.0
379869,2023-05-01,39822,PJ,5358.0,18.0


In [26]:
data24 = data[(data['cliente_id'] == 39822)]
data24

,referencia,cliente_id,tipo_cliente,consumo_kwh,N
379852,2021-12-01,39822,PJ,6246.0,18.0
379853,2022-01-01,39822,PJ,4668.0,18.0
379854,2022-02-01,39822,PJ,4615.0,18.0
379855,2022-03-01,39822,PJ,5480.0,18.0
379856,2022-04-01,39822,PJ,4803.0,18.0
379857,2022-05-01,39822,PJ,5392.0,18.0
379858,2022-06-01,39822,PJ,4762.0,18.0
379859,2022-07-01,39822,PJ,4786.0,18.0
379860,2022-08-01,39822,PJ,4608.0,18.0
379861,2022-09-01,39822,PJ,4513.0,18.0


In [28]:
uc_cliente = data24['cliente_id'].unique()
uc_cliente

array([39822], dtype=int64)

In [29]:
data24 = data24[['referencia', 'consumo_kwh']]
data24 = data24.set_index(['referencia'])
data24.head()

,consumo_kwh
referencia,
2021-12-01,6246.0
2022-01-01,4668.0
2022-02-01,4615.0
2022-03-01,5480.0
2022-04-01,4803.0


In [30]:
TEST_SIZE = 1
train, test = data24.iloc[:-TEST_SIZE], data24.iloc[-TEST_SIZE:]
x_train, x_test = np.array(range(train.shape[0])), np.array(range(train.shape[0], data24.shape[0]))
train.shape, x_train.shape, test.shape, x_test.shape

((17, 1), (17,), (1, 1), (1,))

In [31]:
data24 = data24.reset_index()
data24['Tipo'] = 'Real'
data24['Var_pct'] = 0.0
data24['MAPE'] = 0.0
data24['RMSE'] = 0.0
data24['MAE'] = 0.0
data24['uc'] = uc_cliente[0]
data24.head()

,referencia,consumo_kwh,Tipo,Var_pct,MAPE,RMSE,MAE,uc
0,2021-12-01,6246.0,Real,0.0,0.0,0.0,0.0,39822
1,2022-01-01,4668.0,Real,0.0,0.0,0.0,0.0,39822
2,2022-02-01,4615.0,Real,0.0,0.0,0.0,0.0,39822
3,2022-03-01,5480.0,Real,0.0,0.0,0.0,0.0,39822
4,2022-04-01,4803.0,Real,0.0,0.0,0.0,0.0,39822


In [32]:
model = auto_arima(train, start_p=1, start_q=1,
                      test='adf',
                      max_p=5, max_q=5,
                      m=1,             
                      d=1,          
                      seasonal=False,   
                      start_P=0, 
                      D=None, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=255.394, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=249.535, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=253.647, Time=0.00 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=245.723, Time=0.05 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=247.624, Time=0.09 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.06 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.14 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=243.739, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=247.562, Time=0.02 sec
 ARIMA(3,1,0)(0,0,0)[0]             : AIC=245.644, Time=0.07 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=inf, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=inf, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec

Best mode

In [33]:
data24.tail()

,referencia,consumo_kwh,Tipo,Var_pct,MAPE,RMSE,MAE,uc
13,2023-01-01,4771.0,Real,0.0,0.0,0.0,0.0,39822
14,2023-02-01,4910.0,Real,0.0,0.0,0.0,0.0,39822
15,2023-03-01,5672.0,Real,0.0,0.0,0.0,0.0,39822
16,2023-04-01,4986.0,Real,0.0,0.0,0.0,0.0,39822
17,2023-05-01,5358.0,Real,0.0,0.0,0.0,0.0,39822


In [34]:
# Forecast

prediction, confint = model.predict(n_periods=TEST_SIZE, return_conf_int=True)

prediction

2023-05-01    5210.343806
Freq: MS, dtype: float64

In [35]:
b = prediction.reset_index()
b

,index,0
0,2023-05-01,5210.343806


In [36]:
b['Tipo'] = 'Previsto'
b['Var_pct'] = ((b[0] - data24.loc[17, 'consumo_kwh']) / data24.loc[17, 'consumo_kwh']) * 100
b['MAPE'] = mean_absolute_percentage_error(test.consumo_kwh, prediction)
b['RMSE'] = mean_squared_error(test.consumo_kwh, prediction, squared=False)
b['MAE'] = mean_absolute_error(test.consumo_kwh, prediction)
b['uc'] = uc_cliente[0]
b = b.rename(columns={'index': 'referencia', 0: 'consumo_kwh'})
b

,referencia,consumo_kwh,Tipo,Var_pct,MAPE,RMSE,MAE,uc
0,2023-05-01,5210.343806,Previsto,-2.755808,0.027558,147.656194,147.656194,39822


In [37]:
data24.tail()

,referencia,consumo_kwh,Tipo,Var_pct,MAPE,RMSE,MAE,uc
13,2023-01-01,4771.0,Real,0.0,0.0,0.0,0.0,39822
14,2023-02-01,4910.0,Real,0.0,0.0,0.0,0.0,39822
15,2023-03-01,5672.0,Real,0.0,0.0,0.0,0.0,39822
16,2023-04-01,4986.0,Real,0.0,0.0,0.0,0.0,39822
17,2023-05-01,5358.0,Real,0.0,0.0,0.0,0.0,39822


In [38]:
data24 = pd.concat([data24, b])
data24.tail(), data24.shape

(   referencia  consumo_kwh      Tipo   Var_pct      MAPE        RMSE  \
 14 2023-02-01  4910.000000      Real  0.000000  0.000000    0.000000   
 15 2023-03-01  5672.000000      Real  0.000000  0.000000    0.000000   
 16 2023-04-01  4986.000000      Real  0.000000  0.000000    0.000000   
 17 2023-05-01  5358.000000      Real  0.000000  0.000000    0.000000   
 0  2023-05-01  5210.343806  Previsto -2.755808  0.027558  147.656194   
 
            MAE     uc  
 14    0.000000  39822  
 15    0.000000  39822  
 16    0.000000  39822  
 17    0.000000  39822  
 0   147.656194  39822  ,
 (19, 8))